In [1]:
# Notebook: Host Selection Prototype (final per your spec; no codon usage)

import pandas as pd
import re

# ---------- Load host features ----------
hosts = pd.read_csv("C:\\Users\\solei\\Downloads\\example_hosts_updated.csv")

def split_list(x):
    if pd.isna(x): return []
    return [t.strip().lower() for t in str(x).split(';') if t.strip()]

hosts['AvailableCofactors'] = hosts['AvailableCofactors'].apply(split_list)
hosts['ProteaseProfile']    = hosts['ProteaseProfile'].apply(split_list)
hosts['SupportedSignalPeptides'] = hosts['SupportedSignalPeptides'].apply(split_list)

# ---------- AA feature extraction ----------
KD = {'A':1.8,'C':2.5,'D':-3.5,'E':-3.5,'F':2.8,'G':-0.4,'H':-3.2,'I':4.5,'K':-3.9,'L':3.8,
      'M':1.9,'N':-3.5,'P':-1.6,'Q':-3.5,'R':-4.5,'S':-0.8,'T':-0.7,'V':4.2,'W':-0.9,'Y':-1.3}

def gravy(seq: str) -> float:
    s = seq.upper()
    return sum(KD.get(a,0) for a in s)/max(1,len(s))

def count_n_glyc(seq: str) -> int:
    return len(re.findall(r'N(?!P).[ST]', seq.upper()))

def disulfide_potential(seq: str) -> dict:
    s = seq.upper()
    pos = [i for i,a in enumerate(s) if a == 'C']
    gaps = [j-i for i,j in zip(pos[:-1], pos[1:])]
    return {'CysCount': len(pos), 'CysSpacing': (sum(gaps)/len(gaps) if gaps else None)}

def length_bucket(seq: str) -> str:
    n = len(seq)
    if n < 150: return 'small'
    if n < 600: return 'medium'
    if n < 1200: return 'large'
    return 'very_large'

def pI_proxy(seq: str) -> float:
    s = seq.upper()
    acidic = s.count('D') + s.count('E')
    basic  = s.count('K') + s.count('R') + s.count('H')
    ratio = abs(acidic - basic)/max(1,len(s))
    return max(0.0, 1.0 - 5*ratio)

# ---------- Protease motifs (simple placeholders) ----------
PROTEASE_MOTIFS = {
    'trypsin':    [r'(?<=.)(K|R)(?!P)'],
    'subtilisin': [r'[ASTVILMFYWC]'],
    'cathepsin':  [r'[FYWL]'],
    'yapsins':    [r'KR']
}
def protease_cleavage_counts(seq: str) -> dict:
    s = seq.upper()
    return {p: sum(len(re.findall(pat, s)) for pat in pats) for p, pats in PROTEASE_MOTIFS.items()}

# ---------- Signal peptide detection (presence + type) ----------
# Returns: 'tat' | 'sec' | 'er' | None
def detect_tat(seq: str) -> bool:
    n = seq[:35].upper()
    return re.search(r'[ST]RR.FLK', n) is not None

def has_basic_n_region(s: str, k=1):  # >=1 K/R in first 8 aa
    return sum(1 for a in s[:8] if a in 'KR') >= k

def hydrophobic_core_len(s: str):
    return sum(1 for a in s if KD.get(a,0) > 0)

def has_axA_rule(c_region: str):
    return re.search(r'A.A', c_region) is not None

def detect_signal_peptide_type(seq: str) -> str | None:
    s = seq.upper()
    if detect_tat(s):
        return 'tat'
    n_region = s[:8]; h_region = s[8:25]; c_region = s[15:30]
    if s.startswith('M') and has_basic_n_region(n_region) and hydrophobic_core_len(h_region) >= 8 and has_axA_rule(c_region):
        acidic_n = sum(1 for a in n_region if a in 'DE')
        return 'er' if acidic_n >= 2 else 'sec'
    return None

# ---------- Matching & scoring ----------
def map_size_compat(protein_size_bucket: str, host_size_tol: str) -> float:
    order = ['small','medium','large','very_large']
    p = order.index(protein_size_bucket)
    h = order.index(str(host_size_tol).lower()) if str(host_size_tol).lower() in order else 1
    diff = max(0, p - h)
    return max(0.0, 1.0 - 0.4*diff)

def map_glyco_compat(n_glyc_sites: int, host_glyco: str) -> float:
    g = str(host_glyco).lower()
    if n_glyc_sites == 0: return 1.0
    if g in ['complex','humanized']: return 1.0
    if g == 'high_mannose': return 0.7
    if g == 'none': return 0.3
    return 0.5

def map_disulfide_compat(cys_info: dict, host_disulfide_support: int, secretion_machinery: str) -> float:
    c = cys_info['CysCount']
    if c < 2: return 1.0
    if int(host_disulfide_support) == 1: return 1.0
    sm = str(secretion_machinery).lower()
    if 'tat' in sm or 'er' in sm: return 0.8
    return 0.4

def map_gravy_compat(gravy_val: float, chaperone_strength: str) -> float:
    high = gravy_val >= 0.4
    cs = str(chaperone_strength).lower()
    if high:
        return 1.0 if cs == 'strong' else 0.0
    return 1.0

def scan_cofactor_motifs(seq: str) -> set:
    s = seq.upper()
    motifs = {
        'heme': [r'C..C.H', r'C.{2}C.{1}H', r'CXXCH'],
        'zn2+': [r'C..C..C..C', r'H.E.H'],
        'fe2+': [r'C..C', r'H..H'],
        'fad':  [r'G.G..G', r'VIGAGISG'],
        'ca2+': [r'DD', r'D[DN]D']
    }
    hits = set()
    for cof, pats in motifs.items():
        if any(re.search(p.replace('X','.'), s) for p in pats):
            hits.add(cof)
    return hits

def map_cofactor_compat(required: set, available: list) -> float:
    if not required: return 1.0
    avail = {a.lower() for a in available}
    return 1.0 if required.issubset(avail) else (0.7 if required & avail else 0.4)

def map_protease_baseline(host_risk_idx: float) -> float:
    return max(0.0, 1.0 - float(host_risk_idx))

def map_protease_sequence_specific(seq_counts: dict, host_proteases: list) -> float:
    hp = {p.lower() for p in host_proteases}
    risk_hits = sum(min(n, 10) for p, n in seq_counts.items() if p in hp)
    return max(0.0, 1.0 - min(0.8, 0.05*risk_hits))

def secretion_score(sp_type: str | None, host_supported: list, secretion_machinery: str) -> float | None:
    if sp_type is None:
        return None
    supp = set(host_supported)
    if sp_type == 'er' and (('er' in supp) or ('alpha-factor' in supp)):
        return 1.0
    if sp_type in supp:
        return 1.0
    return 0.0

def score_protein_host(seq, host_row, weights=None):
    if weights is None:
        weights = {
            'Secretion':1, 'Glyco':1, 'Disulfide':1, 'Size':1,
            'GRAVY':1, 'pI':1, 'Cofactor':1, 'Protease':1,
            'GeneralHost':1
        }

    n_glyc  = count_n_glyc(seq)
    cinfo   = disulfide_potential(seq)
    size_b  = length_bucket(seq)
    gval    = gravy(seq)
    pival   = pI_proxy(seq)
    cof_req = scan_cofactor_motifs(seq)
    prot_m  = protease_cleavage_counts(seq)
    sp_type = detect_signal_peptide_type(seq)

    sec_mach  = str(host_row.get('SecretionMachinery','')).lower()
    glyco     = str(host_row.get('GlycoType','none')).lower()
    dis_sup   = int(host_row.get('DisulfideSupport',0))
    size_tol  = str(host_row.get('SizeTolerance','medium')).lower()
    chap      = str(host_row.get('ChaperoneStrength','moderate')).lower()
    cof_avail = host_row.get('AvailableCofactors', [])
    pr_prof   = host_row.get('ProteaseProfile', [])
    pr_idx    = float(host_row.get('ProteaseRiskIndex', 0.5))
    sp_supported = host_row.get('SupportedSignalPeptides', [])

    features = {}
    features['Secretion']   = secretion_score(sp_type, sp_supported, sec_mach)
    features['Glyco']       = map_glyco_compat(n_glyc, glyco)
    features['Disulfide']   = map_disulfide_compat(cinfo, dis_sup, sec_mach)
    features['Size']        = map_size_compat(size_b, size_tol)
    features['GRAVY']       = map_gravy_compat(gval, chap)
    features['pI']          = pival
    features['Cofactor']    = map_cofactor_compat(cof_req, cof_avail)

    p_base = map_protease_baseline(pr_idx)
    p_seq  = map_protease_sequence_specific(prot_m, pr_prof)
    features['Protease']    = min(p_base, p_seq)

    def norm_cat(v, m): return m.get(str(v).lower(), 0.7)
    acc_map = {'low':0.4,'moderate':0.7,'high':1.0}
    grw_map = {'slow':0.4,'moderate':0.7,'fast':1.0}
    rob_map = {'low':0.4,'moderate':0.7,'high':1.0}

    general_protease = map_protease_baseline(pr_idx)
    general = 0.22*norm_cat(host_row.get('GeneticAccessibility'), acc_map) + \
              0.22*norm_cat(host_row.get('SubstrateFlexibility'), acc_map) + \
              0.18*norm_cat(host_row.get('GrowthRate'), grw_map) + \
              0.18*norm_cat(host_row.get('Robustness'), rob_map) + \
              0.10*(1.0 if int(host_row.get('Regulatory',0))==1 else 0.6) + \
              0.10*(1.0 if int(host_row.get('Availability',0))==1 else 0.6)

    general = 0.9*general + 0.1*general_protease
    features['GeneralHost'] = general

    applied = {k:v for k,v in features.items() if v is not None}
    denom = sum(weights[k] for k in applied)
    final = (sum(applied[k]*weights[k] for k in applied) / denom) if denom>0 else 0.0

    features['FinalScore'] = final
    features.update({
        'DetectedSignalPeptide': sp_type if sp_type else '',
        'NGlycSites': n_glyc,
        'CysCount': cinfo['CysCount'],
        'CysSpacing': cinfo['CysSpacing'],
        'SizeBucket': size_b,
        'GRAVYval': gval,
        'pIproxy': pival,
        'CofactorRequired': ';'.join(sorted(cof_req)) if cof_req else '',
        'ProteaseSites_Total': sum(prot_m.values())
    })
    return features

# ---------- UPDATED: Ranking / output ----------
def rank_hosts_for_protein(name, seq, top_n=10, weights=None):
    rows = []
    for _, host in hosts.iterrows():
        sc = score_protein_host(seq, host, weights=weights)
        row = {'Protein': name, 'Host': host['Host']}
        row.update(sc)
        rows.append(row)

    df = pd.DataFrame(rows).sort_values(
        ['FinalScore', 'GeneralHost', 'Host'],
        ascending=[False, False, True],
        kind='mergesort'
    )

    out = df.head(top_n).copy().reset_index(drop=True)

    # Add MatchPct with % sign
    out['MatchPct'] = out['FinalScore'].apply(lambda v: f"{v*100:.1f}%")

    # Move MatchPct to first, FinalScore to last
    preferred = ['MatchPct', 'Protein', 'Host']
    cols = [c for c in preferred if c in out.columns] + \
           [c for c in out.columns if c not in preferred and c != 'FinalScore'] + \
           ['FinalScore']
    out = out[cols]

    filename = f"results_{name.replace(' ', '_')}.csv"
    out.to_csv(filename, index=False)
    print(f"Saved: {filename}")

    try:
        return out.style.hide(axis="index")
    except Exception:
        try:
            return out.style.hide_index()
        except Exception:
            return out

# ---------- Example usage ----------
proteins = {
    'Myoglobin': 'MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRFKHLKSEDEMKASEDLKKHGTVVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPETL',
    'Alpha_casein': 'MSSSSSSTVSSSSSSTPSSVSSSSSTVSSSSSSTVSAPSSVSSSSSSTVSSSSSTVSSSSS',
    'Myofibrillar': 'MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNLHGLFGRKTGQAPGYSYTAANKNKGIIWGEDTL'
}
for name, seq in proteins.items():
    print(f"\nTop matches for {name}")
    display(rank_hosts_for_protein(name, seq, top_n=5))



Top matches for Myoglobin
Saved: results_Myoglobin.csv


MatchPct,Protein,Host,Secretion,Glyco,Disulfide,Size,GRAVY,pI,Cofactor,Protease,GeneralHost,DetectedSignalPeptide,NGlycSites,CysCount,CysSpacing,SizeBucket,GRAVYval,pIproxy,CofactorRequired,ProteaseSites_Total,FinalScore
90.6%,Myoglobin,Corynebacterium_glutamicum,None,1.000000,1.000000,1.000000,1.000000,0.516129,1.000000,0.800000,0.931400,,0,0,None,small,-0.334677,0.516129,fe2+,103,0.905941
87.3%,Myoglobin,Ogataea_polymorpha,None,1.000000,1.000000,1.000000,1.000000,0.516129,1.000000,0.650000,0.821000,,0,0,None,small,-0.334677,0.516129,fe2+,103,0.873391
87.1%,Myoglobin,Pichia_pastoris,None,1.000000,1.000000,1.000000,1.000000,0.516129,1.000000,0.600000,0.852000,,0,0,None,small,-0.334677,0.516129,fe2+,103,0.871016
87.1%,Myoglobin,Saccharomyces_cerevisiae,None,1.000000,1.000000,1.000000,1.000000,0.516129,1.000000,0.600000,0.852000,,0,0,None,small,-0.334677,0.516129,fe2+,103,0.871016
76.1%,Myoglobin,Bacillus_subtilis,None,1.000000,1.000000,1.000000,1.000000,0.516129,0.400000,0.300000,0.870600,,0,0,None,small,-0.334677,0.516129,fe2+,103,0.760841



Top matches for Alpha_casein
Saved: results_Alpha_casein.csv


MatchPct,Protein,Host,Secretion,Glyco,Disulfide,Size,GRAVY,pI,Cofactor,Protease,GeneralHost,DetectedSignalPeptide,NGlycSites,CysCount,CysSpacing,SizeBucket,GRAVYval,pIproxy,CofactorRequired,ProteaseSites_Total,FinalScore
96.6%,Alpha_casein,Corynebacterium_glutamicum,None,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.800000,0.931400,,0,0,None,small,-0.155738,1.000000,,59,0.966425
93.4%,Alpha_casein,Ogataea_polymorpha,None,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.650000,0.821000,,0,0,None,small,-0.155738,1.000000,,59,0.933875
93.2%,Alpha_casein,Pichia_pastoris,None,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.600000,0.852000,,0,0,None,small,-0.155738,1.000000,,59,0.931500
93.2%,Alpha_casein,Saccharomyces_cerevisiae,None,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.600000,0.852000,,0,0,None,small,-0.155738,1.000000,,59,0.931500
89.6%,Alpha_casein,Bacillus_subtilis,None,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.300000,0.870600,,0,0,None,small,-0.155738,1.000000,,59,0.896325



Top matches for Myofibrillar
Saved: results_Myofibrillar.csv


MatchPct,Protein,Host,Secretion,Glyco,Disulfide,Size,GRAVY,pI,Cofactor,Protease,GeneralHost,DetectedSignalPeptide,NGlycSites,CysCount,CysSpacing,SizeBucket,GRAVYval,pIproxy,CofactorRequired,ProteaseSites_Total,FinalScore
84.7%,Myofibrillar,Ogataea_polymorpha,None,1.000000,1.000000,1.000000,1.000000,0.307692,1.000000,0.650000,0.821000,,0,2,3.000000,small,-0.718462,0.307692,fe2+;heme,47,0.847337
84.5%,Myofibrillar,Pichia_pastoris,None,1.000000,1.000000,1.000000,1.000000,0.307692,1.000000,0.600000,0.852000,,0,2,3.000000,small,-0.718462,0.307692,fe2+;heme,47,0.844962
84.5%,Myofibrillar,Saccharomyces_cerevisiae,None,1.000000,1.000000,1.000000,1.000000,0.307692,1.000000,0.600000,0.852000,,0,2,3.000000,small,-0.718462,0.307692,fe2+;heme,47,0.844962
81.7%,Myofibrillar,Corynebacterium_glutamicum,None,1.000000,0.800000,1.000000,1.000000,0.307692,0.700000,0.800000,0.931400,,0,2,3.000000,small,-0.718462,0.307692,fe2+;heme,47,0.817387
73.5%,Myofibrillar,Bacillus_subtilis,None,1.000000,1.000000,1.000000,1.000000,0.307692,0.400000,0.300000,0.870600,,0,2,3.000000,small,-0.718462,0.307692,fe2+;heme,47,0.734787
